In [ ]:
from kubernetes import config, client
from datetime import timezone, datetime
import pandas as pd
%matplotlib inline

## Authenticate and set up kubectl API

In [ ]:
# Run this cell to make sure kubectl is authenticated. If you get an error, debug w/ kubectl
!kubectl --namespace=prod get pod > /dev/null

In [ ]:
CONTEXT = "gke_binder-prod_us-central1-a_prod-a"
NAMESPACE = "prod"
config.load_kube_config(context=CONTEXT)

# Get list of pods with given label selector in the 'prod' namespace
core_api = client.CoreV1Api()

pods = core_api.list_namespaced_pod(NAMESPACE, label_selector="component=singleuser-server")

## Parse data

In [ ]:
data = []
for pod in pods.items:
    age = datetime.now(timezone.utc) - pod.status.start_time.replace(tzinfo=timezone.utc)
    data.append({'age': age.total_seconds() / 60, 'name': pod.metadata.name})

data = pd.DataFrame(data)

## Analyze!

In [ ]:
QUERY = 'jupyterlab'

print('Num matches: {}'.format(data[data['name'].map(lambda a: QUERY in a)].shape))

ax = data[data['name'].map(lambda a: 'jupyterlab' in a)]['age'].plot.hist(bins=20)
ax.set(xlim=[0, None], title=QUERY, xlabel='Pod Age');